# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats # used to calculate percentile scores
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

[{'beta': 1.1196043302150194, 'totalCash': 61555000000, 'currentDebt': 145308000000, 'revenue': 383285000000, 'grossProfit': 169148000000, 'totalRevenue': 383285000000, 'EBITDA': 125820000000, 'revenuePerShare': 24.64, 'revenuePerEmployee': 2607380.95, 'debtToEquity': 5.673462491552152, 'profitMargin': 0.2530623426432028, 'enterpriseValue': 2936745881280, 'enterpriseValueToRevenue': 7.66, 'priceToSales': 7.53, 'priceToBook': 46.46, 'forwardPERatio': 28.14954676382952, 'pegRatio': 87.264082325765, 'peHigh': 27.12919028184647, 'peLow': 14.14151744980657, 'week52highDate': '2023-12-14', 'week52lowDate': '2023-01-05', 'putCallRatio': 0.4787638496632631, 'companyName': 'Apple Inc', 'marketcap': 2887212881280, 'week52high': 199.62, 'week52low': 123.74, 'week52highSplitAdjustOnly': 199.62, 'week52highDateSplitAdjustOnly': '2023-12-14', 'week52lowSplitAdjustOnly': 124.76, 'week52lowDateSplitAdjustOnly': '2023-01-05', 'week52change': 0.4965693541872742, 'sharesOutstanding': 15552752000, 'float'

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data[0]['year1ChangePercent']

0.4405907632709885

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:]:
    
    # the first API call is for parsing company symbol and latestPrice
    batch_api_call_url1 = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data1 = requests.get(batch_api_call_url1).json()
    
    # the second API call is for parsing company year1ChangePercent
    batch_api_call_url2 = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data2 = requests.get(batch_api_call_url2).json()
    
    for i in range(len(data1)):
        # Extract symbol and price from the first API response
        symbol = data1[i]['symbol']
        price = data1[i]['latestPrice']
        
        # Extract year one change percent from the second API response
        year_one_change = data2[i]['year1ChangePercent']
        
        # Create a new row with the combined information
        row_to_add = pd.DataFrame([[symbol, price, year_one_change, 'N/A']], columns=my_columns)
    
        final_dataframe = pd.concat([final_dataframe, row_to_add], ignore_index=True)
        
final_dataframe


/var/folders/ft/tp0qzzj940v6rmm3bb_9k_bw0000gn/T/ipykernel_35350/2594184160.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, row_to_add], ignore_index=True)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,133.350,-0.066100,N/A
1,AAL,13.005,0.056604,N/A
2,AAP,61.675,-0.569773,N/A
3,AAPL,184.205,0.440591,N/A
4,ABBV,160.640,0.028874,N/A
...,...,...,...,...
500,YUM,129.500,0.026372,N/A
501,ZBH,120.790,-0.040533,N/A
502,ZBRA,254.405,0.045123,N/A
503,ZION,42.420,-0.057164,N/A


In [7]:
# export the dataframe for future uses (token expired)
final_dataframe.to_csv('final_dataframe_project2.csv', index=False)

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
# when you write 'inplace = True,' it will not only sort it, but modify the original dataframe
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True, drop = True) # drop will drop the original index
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,345,NVDA,477.460,2.297480,N/A
1,275,LB,79.920,2.281287,N/A
2,390,RCL,117.010,1.430508,N/A
3,370,PHM,100.490,1.250610,N/A
4,84,CCL,16.485,1.145161,N/A
5,31,AMD,134.930,1.139571,N/A
6,50,AVGO,1065.840,1.005127,N/A
7,208,GPS,20.080,0.978951,N/A
8,114,CRM,252.565,0.931744,N/A
9,198,GE,124.740,0.915732,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    while True:
        try:
            portfolio_size = input('Enter the value of your portfolio: ')
            val = float(portfolio_size)
            print(val)
            break  # Break out of the loop if the conversion is successful
        except ValueError:
            print("That's not a number! Please try again:")
    
portfolio_input()

Enter the value of your portfolio: 10000000
10000000.0


In [12]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,345,NVDA,477.460,2.297480,418
1,275,LB,79.920,2.281287,2502
2,390,RCL,117.010,1.430508,1709
3,370,PHM,100.490,1.250610,1990
4,84,CCL,16.485,1.145161,12132
5,31,AMD,134.930,1.139571,1482
6,50,AVGO,1065.840,1.005127,187
7,208,GPS,20.080,0.978951,9960
8,114,CRM,252.565,0.931744,791
9,198,GE,124.740,0.915732,1603


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [13]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]


In [29]:
hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    
    # the first API call is for parsing company symbol and latestPrice
    batch_api_call_url1 = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data1 = requests.get(batch_api_call_url1).json()
    
    # the second API call is for parsing company year1ChangePercent
    batch_api_call_url2 = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data2 = requests.get(batch_api_call_url2).json()
    
    for i in range(len(data1)):
        # Extract symbol and price from the first API response
        symbol = data1[i]['symbol']
        price = data1[i]['latestPrice']
        
        # Extract year one change percent from the second API response
        year_one_change = data2[i]['year1ChangePercent']
        month_6_change = data2[i]['month6ChangePercent']
        month_3_change = data2[i]['month3ChangePercent']
        month_1_change = data2[i]['month1ChangePercent']
        
        # Create a new row with the combined information
        row_to_add = pd.DataFrame([[symbol, price, 'N/A', 
                                    year_one_change, 'N/A',
                                    month_6_change, 'N/A',
                                    month_3_change, 'N/A',
                                    month_1_change, 'N/A', 'N/A']], columns=hqm_columns)
    
        hqm_dataframe = pd.concat([hqm_dataframe, row_to_add], ignore_index=True)
        
hqm_dataframe

/var/folders/ft/tp0qzzj940v6rmm3bb_9k_bw0000gn/T/ipykernel_35350/3185618550.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe = pd.concat([hqm_dataframe, row_to_add], ignore_index=True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.950,N/A,-0.066100,N/A,0.158128,N/A,0.253244,N/A,0.079158,N/A,N/A
1,AAL,13.055,N/A,0.056604,N/A,-0.250836,N/A,0.054118,N/A,0.032258,N/A,N/A
2,AAP,62.120,N/A,-0.569773,N/A,-0.111317,N/A,0.154128,N/A,0.145313,N/A,N/A
3,AAPL,184.055,N/A,0.440591,N/A,-0.037829,N/A,0.071248,N/A,-0.029283,N/A,N/A
4,ABBV,160.410,N/A,0.028874,N/A,0.211380,N/A,0.088835,N/A,0.114427,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.390,N/A,0.026372,N/A,-0.059923,N/A,0.036625,N/A,0.013351,N/A,N/A
501,ZBH,120.890,N/A,-0.040533,N/A,-0.162800,N/A,0.091654,N/A,0.031043,N/A,N/A
502,ZBRA,254.590,N/A,0.045123,N/A,-0.094142,N/A,0.147027,N/A,0.110936,N/A,N/A
503,ZION,42.355,N/A,-0.057164,N/A,0.682646,N/A,0.334047,N/A,0.151879,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [46]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        
        # Get the value from the 'change_col' for the current row
        score = hqm_dataframe.loc[row, change_col]
        
        # Calculate the percentile score of 'score' against all values in 'change_col'
        percentile = stats.percentileofscore(hqm_dataframe[change_col].dropna(), score)/100
        
        # Assign the percentile to the appropriate 'percentile_col' for the current row
        hqm_dataframe.loc[row, percentile_col] = percentile
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PVH,118.640,1685,0.729034,0.84,0.435050,0.88,0.620332,0.98,0.223684,0.94,98.408624
1,AVGO,1063.180,188,1.005127,0.92,0.269667,0.5,0.311913,0.6,0.177829,0.86,94.815195
2,PHM,100.455,1990,1.250610,0.96,0.312514,0.68,0.389628,0.84,0.117870,0.5,94.507187
3,DHI,147.060,1359,0.696944,0.78,0.237057,0.4,0.414879,0.9,0.145346,0.74,94.096509
4,SLG,43.810,4565,0.512183,0.62,0.597562,0.94,0.302677,0.56,0.148439,0.76,93.480493
5,AMD,135.015,1481,1.139571,0.94,0.216574,0.32,0.341919,0.72,0.141610,0.7,92.505133
6,INTC,47.170,4239,0.871156,0.86,0.449218,0.9,0.356879,0.74,0.092821,0.3,92.197125
7,URI,552.080,362,0.618143,0.68,0.281533,0.58,0.301005,0.54,0.130064,0.62,91.889117
8,COF,131.050,1526,0.454085,0.56,0.221384,0.36,0.396899,0.88,0.159572,0.8,91.478439
9,EXPE,146.040,1369,0.698174,0.8,0.359905,0.74,0.457717,0.96,0.067834,0.12,89.373717


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [47]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PVH,118.640,1685,0.729034,0.84,0.435050,0.88,0.620332,0.98,0.223684,0.94,0.91
1,AVGO,1063.180,188,1.005127,0.92,0.269667,0.5,0.311913,0.6,0.177829,0.86,0.72
2,PHM,100.455,1990,1.250610,0.96,0.312514,0.68,0.389628,0.84,0.117870,0.5,0.745
3,DHI,147.060,1359,0.696944,0.78,0.237057,0.4,0.414879,0.9,0.145346,0.74,0.705
4,SLG,43.810,4565,0.512183,0.62,0.597562,0.94,0.302677,0.56,0.148439,0.76,0.72
5,AMD,135.015,1481,1.139571,0.94,0.216574,0.32,0.341919,0.72,0.141610,0.7,0.67
6,INTC,47.170,4239,0.871156,0.86,0.449218,0.9,0.356879,0.74,0.092821,0.3,0.7
7,URI,552.080,362,0.618143,0.68,0.281533,0.58,0.301005,0.54,0.130064,0.62,0.605
8,COF,131.050,1526,0.454085,0.56,0.221384,0.36,0.396899,0.88,0.159572,0.8,0.65
9,EXPE,146.040,1369,0.698174,0.8,0.359905,0.74,0.457717,0.96,0.067834,0.12,0.655


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [48]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

/var/folders/ft/tp0qzzj940v6rmm3bb_9k_bw0000gn/T/ipykernel_35350/2907490038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PVH,118.640,1685,0.729034,0.84,0.435050,0.88,0.620332,0.98,0.223684,0.94,0.91
1,PHM,100.455,1990,1.250610,0.96,0.312514,0.68,0.389628,0.84,0.117870,0.5,0.745
2,GPS,20.310,9847,0.978951,0.9,1.430313,1.0,0.981259,1.0,0.000997,0.02,0.73
3,SLG,43.810,4565,0.512183,0.62,0.597562,0.94,0.302677,0.56,0.148439,0.76,0.72
4,AVGO,1063.180,188,1.005127,0.92,0.269667,0.5,0.311913,0.6,0.177829,0.86,0.72
5,DHI,147.060,1359,0.696944,0.78,0.237057,0.4,0.414879,0.9,0.145346,0.74,0.705
6,INTC,47.170,4239,0.871156,0.86,0.449218,0.9,0.356879,0.74,0.092821,0.3,0.7
7,FITB,34.170,5853,0.155113,0.1,0.392782,0.82,0.448346,0.94,0.160385,0.82,0.67
8,AMD,135.015,1481,1.139571,0.94,0.216574,0.32,0.341919,0.72,0.141610,0.7,0.67
9,CMA,55.025,3634,-0.099957,0.02,0.373970,0.78,0.422769,0.92,0.188100,0.92,0.66


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [49]:
portfolio_input()

Enter the value of your portfolio: 10000000
10000000.0


In [50]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PVH,118.640,1685,0.729034,0.84,0.435050,0.88,0.620332,0.98,0.223684,0.94,0.91
1,PHM,100.455,1990,1.250610,0.96,0.312514,0.68,0.389628,0.84,0.117870,0.5,0.745
2,GPS,20.310,9847,0.978951,0.9,1.430313,1.0,0.981259,1.0,0.000997,0.02,0.73
3,SLG,43.810,4565,0.512183,0.62,0.597562,0.94,0.302677,0.56,0.148439,0.76,0.72
4,AVGO,1063.180,188,1.005127,0.92,0.269667,0.5,0.311913,0.6,0.177829,0.86,0.72
5,DHI,147.060,1359,0.696944,0.78,0.237057,0.4,0.414879,0.9,0.145346,0.74,0.705
6,INTC,47.170,4239,0.871156,0.86,0.449218,0.9,0.356879,0.74,0.092821,0.3,0.7
7,FITB,34.170,5853,0.155113,0.1,0.392782,0.82,0.448346,0.94,0.160385,0.82,0.67
8,AMD,135.015,1481,1.139571,0.94,0.216574,0.32,0.341919,0.72,0.141610,0.7,0.67
9,CMA,55.025,3634,-0.099957,0.02,0.373970,0.78,0.422769,0.92,0.188100,0.92,0.66


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [51]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [52]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

title_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1,
    'bold': True  # This will make the font bold
})

In [53]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], title_format)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [54]:
writer.close()
#remember to check each column in the output excel!